In [20]:
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os
import matplotlib.pyplot as plt

In [21]:
load_dotenv(find_dotenv())

True

In [22]:
project_root = os.environ.get("PROJECT_ROOT")
data_dir = os.environ.get("DATA_DIR")
dataset_fullpath = os.path.join(project_root, data_dir, "output")
dataset_fullpath

'/home/tdy245/ResearchProjects/Winter_2024/CMPT-828/A_Implementation/PaCo/data/output'

In [23]:
large_patches_df = pd.read_csv(os.path.join(dataset_fullpath, "large-patches.csv"))
small_patches_df = pd.read_csv(os.path.join(dataset_fullpath, "small-patches.csv")) 

# Implementation on the small_patches dataset

In [39]:
f"Total number of patches:  {small_patches_df.shape[0]}"

'Total number of patches:  1183'

**Chi-Square Test of Independence**
- Dataset vs Patch Correctness
- APR Tool vs Pacth Correctness

In [42]:
tofI_dataset = pd.crosstab(small_patches_df["dataset"], small_patches_df["label"])

tofI_dataset

label,correct,overfitting
dataset,,
Defects4J-Chart-1,3,14
Defects4J-Chart-10,1,0
Defects4J-Chart-11,3,0
Defects4J-Chart-12,2,5
Defects4J-Chart-13,1,15
...,...,...
Defects4J-Time-5,1,0
Defects4J-Time-6,1,0
Defects4J-Time-7,1,1


In [45]:
f"Total number of unique bugs: {tofI_dataset.shape[0]}"

'Total number of unique bugs: 356'

In [76]:
df = tofI_dataset[['correct', 'overfitting']]

In [77]:
arr = df.to_numpy()

In [79]:
from statsmodels.stats.contingency_tables import mcnemar

result = mcnemar(arr, exact=True)

result.statistic, result.pvalue

(1.0, 0.0009765625)

In [25]:
large_patches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49694 entries, 0 to 49693
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  49694 non-null  object
 1   tool     49694 non-null  object
 2   buggy    49694 non-null  object
 3   patch    49694 non-null  object
 4   label    49694 non-null  object
dtypes: object(5)
memory usage: 1.9+ MB


In [26]:
small_patches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183 entries, 0 to 1182
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  1183 non-null   object
 1   tool     1183 non-null   object
 2   buggy    1183 non-null   object
 3   patch    1183 non-null   object
 4   label    1183 non-null   object
dtypes: object(5)
memory usage: 46.3+ KB


In [27]:

len(large_patches_df["dataset"].unique())

855

In [19]:
large_patches_df["dataset"].unique()

array(['Bugs.jar-math-45', 'Defects4J-Chart-5', 'Defects4J-Math-8',
       'Bears-139', 'Defects4J-Closure-10', 'Bugs.jar-math-24',
       'Defects4J-Closure-13', 'QuixBugs-DEPTH_FIRST_SEARCH-',
       'Bugs.jar-math-33', 'Bugs.jar-math-28', 'Defects4J-Math-20',
       'Bears-126', 'Defects4J-Chart-1', 'Defects4J-Closure-21',
       'Defects4J-Closure-67', 'Defects4J-Lang-7', 'Defects4J-Math-50',
       'Defects4J-Closure-130', 'Defects4J-Closure-59',
       'Defects4J-Math-22', 'Defects4J-Math-31', 'Defects4J-Math-95',
       'Defects4J-Closure-118', 'Bugs.jar-math-105', 'Defects4J-Math-49',
       'Defects4J-Closure-55', 'Defects4J-Closure-22', 'Defects4J-Math-2',
       'Defects4J-Closure-46', 'Defects4J-Math-40',
       'Defects4J-Closure-124', 'Defects4J-Closure-112',
       'Defects4J-Closure-127', 'Defects4J-Closure-117',
       'Defects4J-Math-28', 'Defects4J-Closure-115', 'Defects4J-Math-85',
       'Defects4J-Closure-75', 'Defects4J-Chart-12',
       'Defects4J-Closure-129', 

In [ ]:
len(small_patches_df["tool"].unique())

29